In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
print(tf.config.list_physical_devices('GPU'))

2023-08-29 15:35:59.792843: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-29 15:36:00.561703: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-29 15:36:00.566241: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-29 15:36:06.374757: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[]


2023-08-29 15:36:18.267345: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.expand_dims(x_train, -1).astype("float32") / 255
x_test = np.expand_dims(x_test, -1).astype("float32") / 255
num_classes = 10

In [3]:
inputs = keras.Input(shape =(28,28,1))
x = layers.Conv2D(32,5,strides = (2,2),activation="relu", padding="same")(inputs)
x = layers.Conv2D(32,5,activation="relu", padding="same")(x)
x = layers.MaxPooling2D()(x)
x = layers.Dropout(0.25)(x)
x = layers.Conv2D(64,3,strides = (2,2), activation = "relu", padding = "same")(x)
x = layers.Conv2D(64,3, activation = "relu", padding = "same")(x)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Dropout(0.25)(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation = "softmax")(x)


model = keras.Model(inputs,outputs, name = "CNN_MNIST")
model.summary()

Model: "CNN_MNIST"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 14, 14, 32)        832       
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        25632     
                                                                 
 max_pooling2d (MaxPooling2  (None, 7, 7, 32)          0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 7, 7, 32)          0         
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 64)          18496     
                                                         

2023-08-22 12:06:37.481357: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 222 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:1d:00.0, compute capability: 7.0


In [4]:
model.compile(optimizer = keras.optimizers.Adam(0.001),
              loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics = [keras.metrics.SparseCategoricalAccuracy()]
             )

In [6]:
model.fit(x_train,y_train, epochs = 5, validation_data = [x_test, y_test], batch_size = 10)

2023-08-22 12:09:17.593905: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 179.44MiB (rounded to 188160000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-08-22 12:09:17.593944: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-08-22 12:09:17.593969: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 32, Chunks in use: 32. 8.0KiB allocated for chunks. 8.0KiB in use in bin. 1.4KiB client-requested in use in bin.
2023-08-22 12:09:17.593974: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 1, Chunks in use: 1. 512B allocated for chunks. 512B in use in bin. 512B client-requested in use in bin.
2023-08-22 12:09:17.593978: I tensorflow/tsl/framework/bfc_all

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(x_test[1])
print(y_test[1])
prediction = model.predict(x_test)

In [ ]:
index = 250
plt.imshow(x_test[index])
print(y_test[np.argmax(prediction[index])])

In [ ]:
#%% Save model to SavedModel format
tf.saved_model.save(model, "./models/MNIST_model")

 # load the saved_model using low-level API
model_path = "models/MNIST_model"
m = tf.saved_model.load(model_path)

from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

tfm = tf.function(lambda x: m(x))  # full model                                                  
tfm = tfm.get_concrete_function(tf.TensorSpec(m.signatures['serving_default'].inputs[0].shape.as_list(), m.signatures['serving_default'].inputs[0].dtype.name))   
frozen_func = convert_variables_to_constants_v2(tfm)                                                                                                                              
tf.io.write_graph(graph_or_graph_def=frozen_func.graph, logdir="./", name="MNIST.pb", as_text=False)